In [1]:
import pandas as pd
df=pd.read_csv("C:/Users/dheve/OneDrive/Desktop/ml_assignment/Recommendation System/Recommendation System/anime.csv")

In [2]:
df.columns

Index(['anime_id', 'name', 'genre', 'type', 'episodes', 'rating', 'members'], dtype='object')

In [3]:
df.isnull().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [4]:
df['genre']=df['genre'].fillna(df['genre'].mean)
df['rating']=df['rating'].fillna(df['rating'].mean)
df['type']=df['type'].fillna(df['type'].mean)

In [5]:
df.isnull().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

In [6]:
# Convert columns to numeric, replacing errors (like 'Unknown') with NaN
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')  # 'coerce' converts errors to NaN
df['episodes'] = pd.to_numeric(df['episodes'], errors='coerce')  # 'coerce' converts errors to NaN
df['members'] = pd.to_numeric(df['members'], errors='coerce')  # 'coerce' converts errors to NaN

# Check for any non-numeric data that has been converted to NaN
df.isnull().sum()


anime_id      0
name          0
genre         0
type          0
episodes    340
rating      230
members       0
dtype: int64

In [7]:
# Option 1: Fill missing values with the mean (or median)
df['rating'].fillna(df['rating'].mean(), inplace=True)
df['episodes'].fillna(df['episodes'].mean(), inplace=True)
df['members'].fillna(df['members'].mean(), inplace=True)

# Option 2: Alternatively, drop rows with missing values (if you prefer this approach)
# df.dropna(inplace=True)

# Check if there are any remaining NaN values
df.isnull().sum()


C:\Users\dheve\AppData\Local\Temp\ipykernel_4596\622526958.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['rating'].fillna(df['rating'].mean(), inplace=True)
C:\Users\dheve\AppData\Local\Temp\ipykernel_4596\622526958.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

In [8]:
from sklearn.preprocessing import MinMaxScaler

# Normalize the numerical columns (episodes, rating, members)
scaler = MinMaxScaler()
df[['episodes', 'rating', 'members']] = scaler.fit_transform(df[['episodes', 'rating', 'members']])

# Verify the normalization
df[['episodes', 'rating', 'members']].head()


episodes    rating   members
0  0.000000  0.924370  0.197872
1  0.034673  0.911164  0.782770
2  0.027518  0.909964  0.112689
3  0.012658  0.900360  0.664325
4  0.027518  0.899160  0.149186

In [9]:
# Select only numerical columns for cosine similarity
features = df[['episodes', 'rating', 'members']]

# Check the first few rows to ensure only numeric columns are selected
print(features.head())

# Compute cosine similarity now
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(features)


   episodes    rating   members
0  0.000000  0.924370  0.197872
1  0.034673  0.911164  0.782770
2  0.027518  0.909964  0.112689
3  0.012658  0.900360  0.664325
4  0.027518  0.899160  0.149186


In [10]:
# One-Hot Encoding for 'genre' and 'type'
df_encoded = pd.get_dummies(df[['genre', 'type']], drop_first=True)

# Now concatenate these encoded columns with the numerical ones
features = pd.concat([df[['episodes', 'rating', 'members']], df_encoded], axis=1)

# Verify the new features dataframe
print(features.head())

# Compute cosine similarity
cosine_sim = cosine_similarity(features)


   episodes    rating   members  genre_Action  genre_Action, Adventure  \
0  0.000000  0.924370  0.197872         False                    False   
1  0.034673  0.911164  0.782770         False                    False   
2  0.027518  0.909964  0.112689         False                    False   
3  0.012658  0.900360  0.664325         False                    False   
4  0.027518  0.899160  0.149186         False                    False   

   genre_Action, Adventure, Cars, Comedy, Sci-Fi, Shounen  \
0                                              False        
1                                              False        
2                                              False        
3                                              False        
4                                              False        

   genre_Action, Adventure, Cars, Mecha, Sci-Fi, Shounen, Sports  \
0                                              False               
1                                              False

In [11]:
# Check the unique values in 'genre' to identify problematic entries
print(df['genre'].unique())



['Drama, Romance, School, Supernatural'
 'Action, Adventure, Drama, Fantasy, Magic, Military, Shounen'
 'Action, Comedy, Historical, Parody, Samurai, Sci-Fi, Shounen' ...
 'Hentai, Sports' 'Drama, Romance, School, Yuri' 'Hentai, Slice of Life']


In [12]:
# Fill missing values with 'Unknown'
df['genre'] = df['genre'].fillna('Unknown')

# Check the unique values again to confirm
print(df['genre'].unique())


['Drama, Romance, School, Supernatural'
 'Action, Adventure, Drama, Fantasy, Magic, Military, Shounen'
 'Action, Comedy, Historical, Parody, Samurai, Sci-Fi, Shounen' ...
 'Hentai, Sports' 'Drama, Romance, School, Yuri' 'Hentai, Slice of Life']


In [13]:
# Compute cosine similarity
cosine_sim = cosine_similarity(features)

# Example function to recommend similar anime
def recommend_anime(target_idx, cosine_sim, df, top_n=5, threshold=0.5):
    # Get the cosine similarity scores for the target anime
    sim_scores = list(enumerate(cosine_sim[target_idx]))
    
    # Sort the scores based on similarity (highest first)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the indices of the most similar anime
    similar_anime = [i[0] for i in sim_scores[1:top_n+1]]  # Exclude the target itself (index 0)
    
    # Filter based on a threshold
    similar_anime = [i for i in similar_anime if sim_scores[i][1] >= threshold]
    
    # Get the recommended anime names
    recommended_anime = df['name'].iloc[similar_anime]
    return recommended_anime

# Example: Recommend anime similar to the first anime (index 0) with a threshold of 0.7
recommended = recommend_anime(target_idx=0, cosine_sim=cosine_sim, df=df, top_n=5, threshold=0.7)
print("Recommended Anime:\n", recommended)



Recommended Anime:
 Series([], Name: name, dtype: object)


In [14]:
print(df.columns)


Index(['anime_id', 'name', 'genre', 'type', 'episodes', 'rating', 'members'], dtype='object')


In [16]:
from sklearn.preprocessing import LabelEncoder
# Check the first few rows of the 'type' column to inspect its contents
print(df['type'].head())

# Check if there are any non-string or non-numeric entries in the 'type' column
print(df['type'].apply(type).value_counts())

# If you find any non-standard values (e.g., methods or other objects), you can remove or replace them.
# Here's an example of removing rows where 'type' is an invalid value
df_cleaned = df[df['type'].apply(lambda x: isinstance(x, str))]

# Now try encoding the cleaned 'type' column
label_encoder = LabelEncoder()
df_cleaned['type_encoded'] = label_encoder.fit_transform(df_cleaned['type'])

# Verify the changes
print(df_cleaned[['type', 'type_encoded']].head())


0    Movie
1       TV
2       TV
3       TV
4       TV
Name: type, dtype: object
type
<class 'str'>       12269
<class 'method'>       25
Name: count, dtype: int64
    type  type_encoded
0  Movie             0
1     TV             5
2     TV             5
3     TV             5
4     TV             5


C:\Users\dheve\AppData\Local\Temp\ipykernel_4596\4154338430.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['type_encoded'] = label_encoder.fit_transform(df_cleaned['type'])


In [17]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Assuming df has the structure: user_id, anime_id, rating
# Step 1: Create a user-item matrix
user_item_matrix = df.pivot(index='user_id', columns='anime_id', values='rating').fillna(0)

# Step 2: Calculate user similarity matrix using cosine similarity
user_similarity = cosine_similarity(user_item_matrix)

# Step 3: For a given user (e.g., user_id = 1), find similar users
user_index = user_item_matrix.index.get_loc(1)  # Find index of user_id 1
similar_users = user_similarity[user_index]

# Step 4: Get animes liked by similar users but not yet rated by the target user
similar_users_index = similar_users.argsort()[-3:][::-1]  # Top 3 most similar users
recommendations = user_item_matrix.iloc[similar_users_index].mean(axis=0)

# Sort recommendations and show top recommendations
recommendations = recommendations[recommendations > 0].sort_values(ascending=False)
print("Recommended Animes for User 1:")
print(recommendations.head())


KeyError: 'user_id'

In [18]:
print(df.columns)


Index(['anime_id', 'name', 'genre', 'type', 'episodes', 'rating', 'members'], dtype='object')
